### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{},"{'env': {'game': 'Alien', 'seed': 42, 'type': ...",1be4resa
1,"{'act_acc': 0.6967789174318314, 'pos_sim': 0.9...","{'env': {'game': 'Hero', 'type': 'atari', 'fra...",257y3lo3
2,"{'act_f1': 0.13310726840077225, 'idm_acc': 0.0...","{'env': {'game': 'Frostbite', 'type': 'atari',...",1r3e6wtl
3,"{'_runtime': 8572, 'idm_loss': 2.8987455635070...","{'env': {'game': 'Jamesbond', 'type': 'atari',...",2mpxvuub
4,"{'reward_ratio': 0.0887544377218861, 'loss': 1...","{'env': {'game': 'Gopher', 'type': 'atari', 'f...",3ruf7rzq
...,...,...,...
1434,"{'pos_neg_diff': 0.87976934150327, 'positive_s...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1435,"{'loss': -0.9878902941942216, '_step': 117001,...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1436,"{'_wandb': {'runtime': 18405}, '_runtime': 183...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1437,"{'_wandb': {'runtime': 17734}, '_runtime': 177...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'clt_traj_cons_no_proj'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

16


/tmp/ipykernel_59/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
1,"{'act_acc': 0.6967789174318314, 'pos_sim': 0.9...","{'env': {'game': 'Hero', 'type': 'atari', 'fra...",257y3lo3,Hero
2,"{'act_f1': 0.13310726840077225, 'idm_acc': 0.0...","{'env': {'game': 'Frostbite', 'type': 'atari',...",1r3e6wtl,Frostbite
3,"{'_runtime': 8572, 'idm_loss': 2.8987455635070...","{'env': {'game': 'Jamesbond', 'type': 'atari',...",2mpxvuub,Jamesbond
4,"{'reward_ratio': 0.0887544377218861, 'loss': 1...","{'env': {'game': 'Gopher', 'type': 'atari', 'f...",3ruf7rzq,Gopher
34,"{'reward_ratio': 0.14690734536726838, '_step':...","{'env': {'game': 'CrazyClimber', 'type': 'atar...",1u4pahbk,CrazyClimber
35,"{'max_grad_norm': 0.7762465246001556, 'mean_gr...","{'env': {'game': 'DemonAttack', 'type': 'atari...",2vwko6xd,DemonAttack
36,"{'act_acc': 0.917808324017822, 'min_grad_norm'...","{'env': {'game': 'Freeway', 'type': 'atari', '...",35asodp6,Freeway
37,"{'neg_sim': 0.0944095887344704, 'idm_loss': 2....","{'env': {'game': 'ChopperCommand', 'type': 'at...",3qidgt2o,ChopperCommand
124,"{'pos_sim': 0.9302198111526364, 'idm_loss': 2....","{'env': {'game': 'BankHeist', 'type': 'atari',...",4ce1zzkt,BankHeist
125,"{'act_acc': 0.8515198537650793, 'pos_neg_diff'...","{'env': {'game': 'BattleZone', 'type': 'atari'...",d5wswm3e,BattleZone


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

Hero 257y3lo3
Frostbite 1r3e6wtl
Jamesbond 2mpxvuub
Gopher 3ruf7rzq
CrazyClimber 1u4pahbk
DemonAttack 2vwko6xd
Freeway 35asodp6
ChopperCommand 3qidgt2o
BankHeist 4ce1zzkt
BattleZone d5wswm3e
Breakout 1u4k07ji
Boxing 21jij50n
Assault 3kd5eec3
Alien 3vuyzhop
Amidar 3m74ckao
Asterix eh4n7c9b


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)